<a href="https://colab.research.google.com/github/yub3r/CoderDataScience/blob/main/ProyectoDS-Crime_Data_2020_24-Millan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset - Departamento de Policía de Los Ángeles (LAPD)
Este conjunto de datos refleja incidentes delictivos en la ciudad de Los Ángeles que se remontan a 2020 - *para este estudio solo tendran en cuenta 10mil resgistros random*. Estos datos se transcriben de informes de delitos originales que se escriben en papel y, por lo tanto, puede haber algunas inexactitudes dentro de los datos. Algunos campos de ubicación con datos faltantes se indican como (0°, 0°). Los campos de dirección solo se proporcionan a la cuadra de cien más cercana para mantener la privacidad. Estos datos son tan precisos como los datos de la base de datos. Tenga en cuenta las preguntas o inquietudes en los comentarios.



Fuente Orginal
https://catalog.data.gov/dataset/crime-data-from-2020-to-present


Descripción de las columnas:
https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data

In [18]:
# Import de Librerías necesarias
import pandas as pd
import numpy as np

In [45]:
# Importamos el csv desde github
csv_url = "https://raw.githubusercontent.com/yub3r/CoderDataScience/refs/heads/main/Crime_Data_from_2020_10K2.csv"

#df_datacrime = pd.read_csv(csv_url, nrows=40)
df_datacrime = pd.read_csv(csv_url)

In [46]:
# Aquí se usa el argumento 'on_bad_lines' para saltar las líneas con errores.
#df_datacrime = pd.read_csv(csv_url, on_bad_lines='skip')

print("\nDataframe cargado desde CSV y Github/raw:\n")
print("(Rows, Cols) Totales")
print(df_datacrime.shape)

df_datacrime.head(11)


Dataframe cargado desde CSV y Github/raw:

(Rows, Cols) Totales
(9999, 28)


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,211507896,04/11/2021 12:00:00 AM,11/07/2020 12:00:00 AM,845,15,N Hollywood,1502,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,NaN,NaN,NaN,7800 BEEMAN AV,NaN,34.2124,-118.4092
1,201516622,10/21/2020 12:00:00 AM,10/18/2020 12:00:00 AM,1845,15,N Hollywood,1521,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230,NaN,NaN,NaN,ATOLL AV,N GAULT,34.1993,-118.4203
2,240913563,12/10/2024 12:00:00 AM,10/30/2020 12:00:00 AM,1240,9,Van Nuys,933,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,NaN,NaN,NaN,14600 SYLVAN ST,NaN,34.1847,-118.4509
3,210704711,12/24/2020 12:00:00 AM,12/24/2020 12:00:00 AM,1310,7,Wilshire,782,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,...,IC,Invest Cont,331,NaN,NaN,NaN,6000 COMEY AV,NaN,34.0339,-118.3747
4,201418201,10/03/2020 12:00:00 AM,09/29/2020 12:00:00 AM,1830,14,Pacific,1454,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420,NaN,NaN,NaN,4700 LA VILLA MARINA,NaN,33.9813,-118.4350
5,240412063,12/11/2024 12:00:00 AM,11/11/2020 12:00:00 AM,1210,4,Hollenbeck,429,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,NaN,NaN,NaN,5300 CRONUS ST,NaN,34.0830,-118.1678
6,240317069,12/16/2024 12:00:00 AM,04/16/2020 12:00:00 AM,1350,3,Southwest,396,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,NaN,NaN,NaN,900 W 40TH PL,NaN,34.0100,-118.2900
7,201115217,10/29/2020 12:00:00 AM,07/07/2020 12:00:00 AM,1400,11,Northeast,1133,2,812,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...,...,AO,Adult Other,812,860.0,NaN,NaN,3000 ACRESITE ST,NaN,34.1107,-118.2589
8,241708596,04/20/2024 12:00:00 AM,03/02/2020 12:00:00 AM,1200,17,Devonshire,1729,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,NaN,NaN,NaN,17700 SIMONDS ST,NaN,34.2763,-118.5210
9,242113813,12/18/2024 12:00:00 AM,09/01/2020 12:00:00 AM,900,21,Topanga,2196,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,NaN,NaN,NaN,20900 MARMORA ST,NaN,34.1493,-118.5886


In [53]:
# Conocemos algunos Stats de nuestro Dataframe
df_datacrime.info()
df_datacrime.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DR_NO           9999 non-null   int64  
 1   Date Rptd       9999 non-null   object 
 2   DATE OCC        9999 non-null   object 
 3   TIME OCC        9999 non-null   int64  
 4   AREA            9999 non-null   int64  
 5   AREA NAME       9999 non-null   object 
 6   Rpt Dist No     9999 non-null   int64  
 7   Part 1-2        9999 non-null   int64  
 8   Crm Cd          9999 non-null   int64  
 9   Crm Cd Desc     9999 non-null   object 
 10  Mocodes         8586 non-null   object 
 11  Vict Age        9999 non-null   int64  
 12  Vict Sex        8677 non-null   object 
 13  Vict Descent    8677 non-null   object 
 14  Premis Cd       9999 non-null   int64  
 15  Premis Desc     9997 non-null   object 
 16  Weapon Used Cd  3486 non-null   float64
 17  Weapon Desc     3486 non-null   o

['DR_NO',
 'Date Rptd',
 'DATE OCC',
 'TIME OCC',
 'AREA',
 'AREA NAME',
 'Rpt Dist No',
 'Part 1-2',
 'Crm Cd',
 'Crm Cd Desc',
 'Mocodes',
 'Vict Age',
 'Vict Sex',
 'Vict Descent',
 'Premis Cd',
 'Premis Desc',
 'Weapon Used Cd',
 'Weapon Desc',
 'Status',
 'Status Desc',
 'Crm Cd 1',
 'Crm Cd 2',
 'Crm Cd 3',
 'Crm Cd 4',
 'LOCATION',
 'Cross Street',
 'LAT',
 'LON']

In [63]:
# Nombre de las columnas
print(f"\nNombres de Columnas: {df_datacrime.columns.to_list()} \n")

# Contamos los NaN por Columnas
# df_datacrime.isna().sum()
print(df_datacrime.isnull().sum())


Nombres de Columnas: ['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON'] 

DR_NO                0
Date Rptd            0
DATE OCC             0
TIME OCC             0
AREA                 0
AREA NAME            0
Rpt Dist No          0
Part 1-2             0
Crm Cd               0
Crm Cd Desc          0
Mocodes           1413
Vict Age             0
Vict Sex          1322
Vict Descent      1322
Premis Cd            0
Premis Desc          2
Weapon Used Cd    6513
Weapon Desc       6513
Status               0
Status Desc          0
Crm Cd 1             0
Crm Cd 2          9174
Crm Cd 3          9966
Crm Cd 4          9998
LOCATION             0
Cross Street      8426
LAT                  0
LON     

# Limpieza
Luego de haber estudiado los resultados de consultas anteriores, procedemos a limpiar columnas que creemos innecesarias y dejamos solo los datos relevantes.